<a href="https://colab.research.google.com/github/caazenrique/sqlonbigquery/blob/main/E_mail_engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
WITH user_metrics AS
# A CTE user_metrics se baseia nas tabelas account, account_session,session e session_params


(
  SELECT
  s.date AS date,
  sp.country AS country,
  acc.send_interval AS send_interval,
  acc.is_verified AS is_verified,
  acc.is_unsubscribed AS is_unsubscribed,
  COUNT(DISTINCT acc.id) AS account_cnt,

FROM DA.account            AS acc
JOIN DA.account_session    AS acs ON acc.id = acs.account_id
JOIN DA.session            AS s   ON acs.ga_session_id = s.ga_session_id
JOIN DA.session_params     AS sp  ON acs.ga_session_id = sp.ga_session_id


GROUP BY
  s.date,
  sp.country,
  acc.send_interval,
  acc.is_verified,
  acc.is_unsubscribed
),


# AS CTEs de email_metrics agregam dados de envio, abertura e clique nos emails todas


email_sent_metrics AS


(
  SELECT
      DATE_ADD(s.date, INTERVAL es.sent_date DAY) AS date,
      sp.country AS country,
      acc.send_interval AS send_interval,
      acc.is_verified AS is_verified,
      acc.is_unsubscribed AS is_unsubscribed,
      COUNT (DISTINCT es.id_account) AS email_sent_cnt

  FROM `DA.account` AS acc
  JOIN `DA.account_session` AS acs ON acc.id = acs.account_id
  JOIN `DA.session` AS s ON s.ga_session_id = acs.ga_session_id
  JOIN `DA.email_sent` AS es ON es.id_account = acc.id
  LEFT JOIN `DA.email_open` AS eo ON es.id_message = eo.id_message
  LEFT JOIN `DA.email_visit` AS ev ON es.id_message = ev.id_message
  JOIN `DA.session_params` AS sp ON acs.ga_session_id = sp.ga_session_id
   GROUP BY
      date,
      sp.country,
      acc.send_interval,
      acc.is_verified,
      acc.is_unsubscribed
),


email_open_metrics AS


(
  SELECT
      DATE_ADD(s.date, INTERVAL es.sent_date DAY) AS date,
      sp.country AS country,
      acc.send_interval AS send_interval,
      acc.is_verified AS is_verified,
      acc.is_unsubscribed AS is_unsubscribed,
      COUNT (DISTINCT eo.id_message) AS email_open_cnt

  FROM `DA.account` AS acc
  JOIN `DA.account_session` AS acs ON acc.id = acs.account_id
  JOIN `DA.session` AS s ON s.ga_session_id = acs.ga_session_id
  JOIN `DA.email_sent` AS es ON es.id_account = acc.id
  LEFT JOIN `DA.email_open` AS eo ON es.id_message = eo.id_message
  LEFT JOIN `DA.email_visit` AS ev ON es.id_message = ev.id_message
  JOIN `DA.session_params` AS sp ON acs.ga_session_id = sp.ga_session_id
  GROUP BY
      date,
      sp.country,
      acc.send_interval,
      acc.is_verified,
      acc.is_unsubscribed
),


email_visit_metrics AS


(
  SELECT
      DATE_ADD(s.date, INTERVAL es.sent_date DAY) AS date,
      sp.country AS country,
      acc.send_interval AS send_interval,
      acc.is_verified AS is_verified,
      acc.is_unsubscribed AS is_unsubscribed,
      COUNT (DISTINCT ev.id_message) AS email_visit_cnt

  FROM `DA.account` AS acc
  JOIN `DA.account_session` AS acs ON acc.id = acs.account_id
  JOIN `DA.session` AS s ON s.ga_session_id = acs.ga_session_id
  JOIN `DA.email_sent` AS es ON es.id_account = acc.id
  LEFT JOIN `DA.email_open` AS eo ON es.id_message = eo.id_message
  LEFT JOIN `DA.email_visit` AS ev ON es.id_message = ev.id_message
  JOIN `DA.session_params` AS sp ON acs.ga_session_id = sp.ga_session_id
  GROUP BY
      date,
      sp.country,
      acc.send_interval,
      acc.is_verified,
      acc.is_unsubscribed
),


# A CTE final_metrics agrega as métricas de usuário e e-mails através do UNION ALL

final_metrics AS
(
SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      account_cnt,
      0 AS email_sent_cnt,
      0 AS email_open_cnt,
      0 AS email_visit_cnt


FROM user_metrics


UNION ALL


SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      0 AS account_cnt,
      email_sent_cnt,
      0 AS email_open_cnt,
      0 AS email_visit_cnt


FROM email_sent_metrics


UNION ALL


SELECT
       date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      0 AS account_cnt,
      0 AS email_sent_cnt,
      email_open_cnt,
      0 AS email_visit_cnt


FROM email_open_metrics


UNION ALL


SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      0 AS account_cnt,
      0 AS email_sent_cnt,
      0 AS email_open_cnt,
      email_visit_cnt,



FROM email_visit_metrics




GROUP BY


      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      account_cnt,
      email_sent_cnt,
      email_open_cnt,
      email_visit_cnt


ORDER BY

        country
),


# A CTE abaixo calcula a soma dos dados numéricos e os valores gerais calculados através de window functions

total_country_data AS


(
SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      SUM(account_cnt) AS account_cnt,
      SUM(email_sent_cnt) AS email_sent_cnt,
      SUM(email_open_cnt) AS email_open_cnt,
      SUM(email_visit_cnt) AS email_visit_cnt,
      SUM(SUM(account_cnt)) OVER (PARTITION BY country) AS total_country_account_cnt,
      SUM(SUM(email_sent_cnt)) OVER (PARTITION BY country) AS total_country_sent_cnt


FROM final_metrics
GROUP BY
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed
),


# Por último são aplicadas as funções que trazem o ranking pelas métricas account_cnt e email_sent_cnt

final_ranking AS


(
SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      account_cnt,
      email_sent_cnt,
      email_open_cnt,
      email_visit_cnt,
      total_country_account_cnt,
      total_country_sent_cnt,
      DENSE_RANK() OVER (ORDER BY total_country_account_cnt DESC) AS rank_total_country_account_cnt,
      DENSE_RANK() OVER (ORDER BY total_country_sent_cnt DESC) AS rank_total_country_sent_cnt




FROM total_country_data




ORDER BY rank_total_country_account_cnt, rank_total_country_sent_cnt
)


SELECT
      date,
      country,
      send_interval,
      is_verified,
      is_unsubscribed,
      account_cnt,
      email_sent_cnt,
      email_open_cnt,
      email_visit_cnt,
      total_country_account_cnt,
      total_country_sent_cnt,
      rank_total_country_account_cnt,
      rank_total_country_sent_cnt


FROM final_ranking
WHERE rank_total_country_account_cnt <= 10 AND rank_total_country_sent_cnt <= 10




ORDER BY rank_total_country_account_cnt, rank_total_country_sent_cnt